In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!git clone https://github.com/realcrane/iPLAN-Interactive-and-Procedural-Layout-Planning

Cloning into 'iPLAN-Interactive-and-Procedural-Layout-Planning'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 301 (delta 96), reused 106 (delta 46), pack-reused 91
Receiving objects: 100% (301/301), 12.98 MiB | 5.41 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [3]:
path_model_roomLocation_resnet = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/resnet18-5c106cde.pth"
path_model_roomLocation_location_up = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/location_up1_100.pth"
path_model_roomLocation_location_resnet = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/location_resnet18_100.pth"
path_model_roomLocation_location_embed = "/content/drive/MyDrive/Colab_Notebooks/Deep_Learning/Progetto/Model/room_location/location_embed_100.pth"

#Location

In [4]:


import os
print("Directory di lavoro corrente:", os.getcwd())
roomType_directory = '/content/iPLAN-Interactive-and-Procedural-Layout-Planning/room_location/Location'
os.chdir(roomType_directory)
print("Nuova directory di lavoro:", os.getcwd())

Directory di lavoro corrente: /content
Nuova directory di lavoro: /content/iPLAN-Interactive-and-Procedural-Layout-Planning/room_location/Location


In [5]:
from config_location import LocationConfig
opt = LocationConfig()

In [6]:
import utils
import models

model = models.model(
        module_name=opt.module_name,
        model_name=opt.model_name,
        input_channel=utils.num_category+4,
        output_channel=utils.num_category+3,
        pretrained=False
    )


input_channel = 512
connect = models.connect(
    module_name=opt.module_name,
    model_name=opt.model_name,
    input_channel=input_channel,
    output_channel=utils.num_category+3,
    reshape=False
)


embedding = models.embedding(
    module_name=opt.module_name,
    model_name=opt.model_name,
    input_channel=13,
    output_channel=256,
    reshape=False
)

model.load_model(path_model_roomLocation_location_resnet)
connect.load_model(path_model_roomLocation_location_up)
embedding.load_model(path_model_roomLocation_location_embed)

print(model)
print('###################')
print(connect)
print('###################')
print(embedding)

ResNet(
  (conv1): Conv2d(17, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01, inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.01, inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [7]:
model.cuda()
connect.cuda()
embedding.cuda()
print()

In [30]:
from torch.utils.data import DataLoader
from dataset import LocationDataset
val_data = LocationDataset(data_root=opt.data_root, mask_size=opt.mask_size, phase='test')

opt.num_workers = 2 #originally is 4
val_dataloader = DataLoader(
        val_data,
        1, #opt.batch_size,
        shuffle=False,
        num_workers=opt.num_workers
    )

In [31]:
import torch as t
import numpy as np

In [32]:
weight = t.ones(utils.num_category+3)
for i in range(utils.num_category+1):
    if i < 13:
        weight[i] = 2
    else:
        weight[i] = 1.25
weight = weight.cuda()
criterion = t.nn.CrossEntropyLoss(weight=weight)

In [34]:
model.eval()
connect.eval()
embedding.eval()
# loss_meter = meter.AverageValueMeter()
# loss_meter.reset()
predict_accuracy = 0
num_predict_Category_right = 0
num_target_Category = 0
num_predict_Category = 0
num_target_specific_category = 0
num_predict_specific_category = 0
num_predict_specific_category_right = 0
softmax = t.nn.Softmax(dim=1)

for _, (input, target, room_type) in enumerate(val_dataloader):
    batch_size = input.shape[0]
    with t.no_grad():
        input = input.cuda()
        room_type = room_type.cuda()
        target = target.cuda()
        score_model = model(input)
        score_embedding = embedding(room_type)
        score_temp = t.cat([score_model, score_embedding], 1)
        score_connect = connect(score_temp)
        # loss = criterion(score_connect, target)
        # loss_meter.add(loss.item())

    score_softmax = softmax(score_connect)
    output = score_softmax.cpu().numpy()
    predict = np.argmax(output, axis=1)
    target = target.cpu().numpy()
    for i in range(batch_size):
        type = t.where(room_type[i,:]==1)[0][0]
        num_predict = np.sum(predict[i] == target[i])
        predict_accuracy += (num_predict / (input.shape[2]*input.shape[3]))
        for k in range(utils.num_category+1):
            num_predict_Category_right += np.sum((predict[i] == k) & (target[i] == k))
            num_target_Category += np.sum(target[i] == k)
            num_predict_Category += np.sum(predict[i] == k)

            if k == type:
                num_target_specific_category += np.sum(target[i] == k)
                num_predict_specific_category += np.sum((predict[i] == k))
                num_predict_specific_category_right += np.sum((predict[i] == k) & (target[i] == k))

In [35]:
batch_size

1

In [36]:
output.shape

(1, 16, 128, 128)

In [51]:
predict.shape

(1, 128, 128)

In [38]:
print(len(target), len(target[0]), len(target[1]))
print(len(predict), len(predict[0]), len(predict[1]))

IndexError: ignored

In [39]:
print("Predict: \n", predict, "\n")
print("Target: \n", target)

Predict: 
 [[[15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]
  ...
  [15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]]] 

Target: 
 [[[15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]
  ...
  [15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]
  [15 15 15 ... 15 15 15]]]


In [ ]:
print(f"Predict_accuracy: {predict_accuracy} \n")

print(f"Num_target_Category: {num_target_Category}")
print("------------------------")
print(f"Num_predict_Category: {num_predict_Category}")
print(f"Num_predict_Category_right: {num_predict_Category_right}")
print(f"Percentage pretected specific category: {(num_target_Category/num_predict_Category): .3g}%")
print(f"Percentage wrong category (false positive): {(num_predict_Category/num_target_Category) - (num_predict_Category_right/num_target_Category): .3g}% ")
print(f"Percentage rigth category (true positive): {num_predict_Category_right/num_target_Category: .3g}% \n \n")


print(f"Num_target_specific_category: {num_target_specific_category}")
print("------------------------")
print(f"Num_predict_specific_category: {num_predict_specific_category}")
print(f"Num_predict_specific_category_right: {num_predict_specific_category_right}")
print(f"Percentage pretected specific category: {num_target_specific_category/num_predict_specific_category: .3g}%")
print(f"Percentage wrong specific category (false positive): {(num_predict_specific_category/num_target_specific_category) - (num_predict_specific_category_right/num_target_specific_category): .3g}% ")
print(f"Percentage rigth specific category (true positive): {num_predict_specific_category_right/num_target_specific_category: .3g}%")

Predict_accuracy: 1.9970703125 

Num_target_Category: 648
------------------------
Num_predict_Category: 666
Num_predict_Category_right: 634
Percentage pretected specific category:  0.973%
Percentage wrong category (false positive):  0.0494% 
Percentage rigth category (true positive):  0.978% 
 

Num_target_specific_category: 162
------------------------
Num_predict_specific_category: 180
Num_predict_specific_category_right: 149
Percentage pretected specific category:  0.9%
Percentage wrong specific category (false positive):  0.191% 
Percentage rigth specific category (true positive):  0.92%


In [53]:
import matplotlib.pyplot as plt

img = t.cat((input[0], input[1]), dim=0)



fig, axes = plt.subplots(nrows=1, ncols=17, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    ax.imshow(img[i].cpu(), cmap='gray', interpolation='none')
    ax.axis('off')

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=17, figsize=(15, 6))
for i, ax in enumerate(axes.flat):

    ax.imshow(img[17 + i].cpu(), cmap='gray', interpolation='none')
    ax.axis('off')

plt.tight_layout()
plt.show()

IndexError: ignored

#Label Target e Predict

### Come si può notare il modello predice perfettamente le label fornite come validation.

In [ ]:
for t,p in target, predict:
  print(f"target: \n{t} \n")
  print(f"predict: \n{p} \n")
  if t.any() == p.any():
    print("I valori forniti sono uguali")

target: 
[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]] 

predict: 
[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]] 

I valori forniti sono uguali
target: 
[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]] 

predict: 
[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]] 

I valori forniti sono uguali


In [61]:
for riga in predict:
  print(riga)
  for val in riga:
    print(val)
    break
  break

[[15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 ...
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]
 [15 15 15 ... 15 15 15]]
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15]


In [64]:
import numpy as np
import matplotlib.pyplot as plt

# Creazione di un tensore 128x128 con valori da 1 a 15

# Definizione della mappatura dei colori per ogni numero
colori = ['white', 'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange', 'gray', 'purple', 'brown', 'pink', 'olive', 'teal', 'navy']
colori_dict = {color: i for i, color in enumerate(colori, 1)}
# Creazione di un'immagine usando la mappatura dei colori
immagine_colorata = np.array([[colori_dict[color] for color in riga] for riga in  predict[0, :, :]])

# Visualizzazione dell'immagine usando Matplotlib
plt.imshow(immagine_colorata)
plt.axis('off')  # Disattiva gli assi
plt.show()


KeyError: ignored